In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (TextVectorization, Dense, MultiHeadAttention, LayerNormalization, 
                                     Layer, Embedding, Input, Dropout)
from tensorflow.keras.callbacks import EarlyStopping

2023-03-02 14:55:12.474241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 14:55:12.641910: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-02 14:55:12.680060: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-02 14:55:13.435912: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (TextVectorization, Dense, MultiHeadAttention, LayerNormalization, 
                                     Layer, Embedding, Input, Dropout)
from tensorflow.keras.callbacks import EarlyStopping

import logging

GPU_FROM = 0
GPU_TO = 1  

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs visible:{len(visible_devices)}")
tf.config.set_visible_devices(visible_devices[GPU_FROM:GPU_TO],'GPU')

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs to be used: {len(visible_devices)}")

# Build dataset

In [3]:
FULL_VOCAB = 'abcdefghijklmnopqrstuvwxyz'
SEQ_LEN = 10

In [4]:
def dataset1(vocab_size=2, dataset_size=10000, seq_len=10, full_vocab=FULL_VOCAB):
  """
  Inserts a space after a the first character in the vocabulary (and nowhere else)
  """
  assert vocab_size > 1
  vocab = list(full_vocab[:vocab_size])
  inputs = np.random.choice(vocab, size=(dataset_size, seq_len))
  outputs = np.where(inputs == vocab[0], 1., 0.).astype(np.float32)  # 1 = space, 0 = no space
  concatenated_inputs = np.array([''.join(row) for row in inputs])
  return concatenated_inputs, outputs

def dataset2(vocab_size=2, dataset_size=10000, seq_len=10, full_vocab=FULL_VOCAB):
  """
  Inserts a space after the combination of 1st->2nd character in the vocabulary (and nowhere else)
  """
  assert vocab_size > 1
  vocab = list(full_vocab[:vocab_size])
  inputs = np.random.choice(vocab, size=(dataset_size, seq_len))
  outputs = np.zeros_like(inputs, dtype=np.float32)
  for i, example in enumerate(inputs):
    previous_char = example[0]
    for j, char in enumerate(example[1:]):
      if (previous_char == vocab[0]) and (char == vocab[1]):  # 1 = space, 0 = no space
        outputs[i, j+1] = 1.
      previous_char = char
  concatenated_inputs = np.array([''.join(row) for row in inputs])
  return concatenated_inputs, outputs

def dataset3(vocab_size=2, dataset_size=10000, seq_len=10, insert_space_every=3, full_vocab=FULL_VOCAB):
  """
  Inserts a space after a certain number of characters, no matter what the characters
  """
  assert vocab_size > 1
  vocab = list(full_vocab[:vocab_size])
  inputs = np.random.choice(vocab, size=(dataset_size, seq_len))
  outputs = np.zeros_like(inputs, dtype=np.float32)
  outputs[:, np.arange(insert_space_every-1, outputs.shape[1], insert_space_every)] = 1.
  concatenated_inputs = np.array([''.join(row) for row in inputs])
  return concatenated_inputs, outputs

In [5]:
DATASET_FN = dataset2

train_ds = tf.data.Dataset.from_tensor_slices(DATASET_FN(vocab_size=2, seq_len=SEQ_LEN))
valid_ds = tf.data.Dataset.from_tensor_slices(DATASET_FN(vocab_size=2, seq_len=SEQ_LEN))
test_ds = tf.data.Dataset.from_tensor_slices(DATASET_FN(vocab_size=2, seq_len=SEQ_LEN))
train_ds.element_spec

2023-03-02 14:55:15.406265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 14:55:16.217637: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22296 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(10,), dtype=tf.float32, name=None))

In [6]:
train_ds = train_ds.shuffle(1000).batch(128)
valid_ds = valid_ds.batch(128)
test_ds = test_ds.batch(128)

In [7]:
for test_inputs, test_outputs in train_ds.take(1):
  pass
print(test_inputs)
tf.print(test_outputs,summarize=-1)

tf.Tensor(
[b'aababbbaaa' b'abbbaaabbb' b'ababbabaab' b'babaaaaaaa' b'baabababab'
 b'bbbbbbbbaa' b'aaabbbbbbb' b'bbabaaaaab' b'babaaaaaab' b'bbabbaabaa'
 b'abbaabbbbb' b'bbbbbbabaa' b'aaabbaaabb' b'bbabbbaaab' b'aaabbbbabb'
 b'aaabababbb' b'ababaaaaaa' b'abababbaaa' b'babbabbbaa' b'bbaaaabbab'
 b'aabbbbabaa' b'baabababab' b'bbbababbab' b'aaaaaabbbb' b'baabaaabaa'
 b'abbbaabbaa' b'bbbaaaabbb' b'bbabbbaaaa' b'bbbaabbaab' b'abaaabbbba'
 b'babababaab' b'bbbbaaaaab' b'bbaaaababa' b'babbbbabba' b'ababbbabba'
 b'bbbabbbbba' b'abaababbab' b'baaaabbbba' b'bbabbabbbb' b'bbaaaababb'
 b'abbabbaabb' b'bbbbbbabbb' b'baaaabbbba' b'abbababbba' b'ababbaaaba'
 b'bbabaababa' b'abbaaabaab' b'aaaaaaabbb' b'bbbbaaaaaa' b'bababababb'
 b'baaabbbaab' b'aaabbaabaa' b'babbaabbbb' b'aabbbbbbaa' b'aababababa'
 b'baaaaabbaa' b'baaaabbbba' b'babaabbaab' b'bbbabbbbbb' b'bababbbaab'
 b'abaaaaabba' b'abbabbbbaa' b'aaabaabbba' b'abaabbbaba' b'aabaabaaaa'
 b'aabbaabbbb' b'babaaaabba' b'aaaaaababa' b'babbbaaaaa' b'abaaaba

# Build layers

In [8]:
# # Embedding defn
def positional_encoding(length, depth):
    """
    Generates a matrix following:
    $$
        PE_{pos,i} = trig(\frac{pos, 10000^{\frac{i, d}})
    $$
    where d is the dimensionality of the output embedding and the position
    is defined absolutely (from 0).
    """
    per_trig_d_model = depth/2
    

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(per_trig_d_model)[np.newaxis, :]/per_trig_d_model   # (1, depth/2)
    angle_rates = 1 / (10000**depths)         # (1, depth/2)
    angle_rads = positions * angle_rates      # (seq, depth/2)

    pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)], # (seq, depth)
      axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

class PostionalEmbedding(tf.keras.layers.Layer):
    # TODO: FIX spelling
    def __init__(
            self,
            vocab_size, 
            d_model, 
            max_seq_len,
            pos_multiplier=1, 
            mask_zero=True):
        """
        Generate a layer to embed input tokens that are already int-encoded
         through a lookup embedding and positional information 
         (through additive positional embeddings as in Vaswani 2017).
        """
        super().__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=mask_zero) 
        self.pos_encoding = positional_encoding(length=max_seq_len, depth=d_model)
        self.supports_masking = True
        self.pos_multiplier = pos_multiplier

    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x, mask=None):
        # Assumes (batch, seq_len) int-encoded inputs
        x = self.embedding(x) # (batch, seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # TODO: try running without this
        x = x + self.pos_multiplier*self.pos_encoding[tf.newaxis, :, :] # new axis for batch dimension - try without
        return x

def point_wise_feed_forward_network(
    d_model, # Input/output dimensionality.
    dff # Inner-layer dimensionality.
    ):

    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
    ])

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,*,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1,
               **kwargs
               ):
        super().__init__(**kwargs)


        # Multi-head self-attention.
        self.mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate, # TODO: maybe dropout?
            )
        # Point-wise feed-forward network.
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        # Layer normalization.
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # Dropout for the point-wise feed-forward network.
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training, mask):

        # A boolean mask.
        if mask is not None:
            mask1 = mask[:, :, None] # (B, seq, 1)
            mask2 = mask[:, None, :] # (B, 1, seq)
            attention_mask = mask1 & mask2 
            # For each element in the sequence - what other elements can it attend to
            # only defined in this simple self-cartesion product because we are
            # simply attempting to mask away unused token slots for efficiency
            # Has block strucure, NOT upper triangular like a causal mask would
        else:
            attention_mask = None

        # Multi-head self-attention output (`tf.keras.layers.MultiHeadAttention `).
        attn_output = self.mha(
            query=x,  # Query Q tensor.
            value=x,  # Value V tensor.
            key=x,  # Key K tensor.
            attention_mask=attention_mask, # A boolean mask that prevents attention to certain positions.
            training=training, # A boolean indicating whether the layer should behave in training mode.
            )

        # Multi-head self-attention output after layer normalization and a residual/skip connection.
        out1 = self.layernorm1(x + attn_output)  # Shape `(batch_size, input_seq_len, d_model)`

        # Point-wise feed-forward network output.
        ffn_output = self.ffn(out1)  # Shape `(batch_size, input_seq_len, d_model)`
        ffn_output = self.dropout1(ffn_output, training=training) # TODO: try removing dropout errywhere
        # Point-wise feed-forward network output after layer normalization and a residual skip connection.
        out2 = self.layernorm2(out1 + ffn_output)  # Shape `(batch_size, input_seq_len, d_model)`.

        return out2

class Encoder(tf.keras.layers.Layer):
    def __init__(self,
               *,
               num_layers,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               tokenizer, # int-mode tokenizer for input text,
               seq_len,
               dropout_rate=0.1,
               pos_multiplier=1 
               ):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        # Tokenization
        self.tokenizer = tokenizer

        # Embeddings + Positional encoding
        self.pos_embedding = PostionalEmbedding(tokenizer.vocabulary_size(), d_model, seq_len, pos_multiplier=pos_multiplier)

        # Encoder layers.
        self.enc_layers = [
            EncoderLayer(
              d_model=d_model,
              num_attention_heads=num_attention_heads,
              dff=dff,
              dropout_rate=dropout_rate,
              name=f"encoder_sublayer_{i}")
            for i in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def compute_mask(self, x, previous_mask=None):
        x = self.tokenizer(x)
        return self.pos_embedding.compute_mask(x, previous_mask)

    def call(self, x, training):
        # Sum up embeddings and positional encoding.
        x = self.tokenizer(x)
        mask = self.pos_embedding.compute_mask(x)
        # TODO: why am i giving back the mask here?
        # FIXME: Remove pointless mask passing
        x = self.pos_embedding(x, mask=mask)  # Shape `(batch_size, input_seq_len, d_model)`.
        # Add dropout.
        # ?
        x = self.dropout(x, training=training)

        # N encoder layers.
        for i in range(self.num_layers):
            # TODO: Do not need to pass mask 
            # TODO: Check mask propagation
            x = self.enc_layers[i](x, training, mask)

        return x  # Shape `(batch_size, input_seq_len, d_model)`.

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,
               *,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1,
               **kwargs
               ):
        super().__init__(**kwargs)

        # Masked multi-head self-attention.
        self.mha_masked = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate
        )
        # Multi-head cross-attention.
        self.mha_cross = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate
        )

        # Point-wise feed-forward network.
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        # Layer normalization.
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # Dropout for the point-wise feed-forward network.
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, mask, enc_output, enc_mask, training):
        # The encoder output shape is `(batch_size, input_seq_len, d_model)`.

        # A boolean mask.
        self_attention_mask = None
        if mask is not None:
            mask1 = mask[:, :, None]
            mask2 = mask[:, None, :]
            self_attention_mask = mask1 & mask2

        # Masked multi-head self-attention output (`tf.keras.layers.MultiHeadAttention`).
        attn_masked, attn_weights_masked = self.mha_masked(
            query=x,
            value=x,
            key=x,
            attention_mask=self_attention_mask,  # A boolean mask that prevents attention to certain positions.
            use_causal_mask=True,  # A boolean to indicate whether to apply a causal mask to prevent tokens from attending to future tokens.
            return_attention_scores=True,  # Shape `(batch_size, target_seq_len, d_model)`.
            training=training  # A boolean indicating whether the layer should behave in training mode.
            )

        # Masked multi-head self-attention output after layer normalization and a residual/skip connection.
        self_attn = self.layernorm1(attn_masked + x)

        # A boolean mask.
        attention_mask = None
        if mask is not None and enc_mask is not None:
            mask1 = mask[:, :, None]
            mask2 = enc_mask[:, None, :]
            attention_mask = mask1 & mask2

        # Multi-head cross-attention output (`tf.keras.layers.MultiHeadAttention `).
        attn_cross, attn_weights_cross = self.mha_cross(
            query=enc_output,
            key=self_attn,
            value=self_attn,
            attention_mask=attention_mask,  # A boolean mask that prevents attention to certain positions.
            return_attention_scores=True,  # Shape `(batch_size, num_queries, d_model)`.
            training=training  # A boolean indicating whether the layer should behave in training mode.
        )

        # Multi-head cross-attention output after layer normalization and a residual/skip connection.
        out2 = self.layernorm2(attn_cross + self_attn)  # (batch_size, source_seq_len, d_model)

        # Point-wise feed-forward network output.
        ffn_output = self.ffn(out2)  # Shape `(batch_size, source_seq_len, d_model)`.
        ffn_output = self.dropout1(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # Shape `(batch_size, source_seq_len, d_model)`.

        return out3


class Decoder(tf.keras.layers.Layer):
    output_tokens = [0, 1, 2]   # We only have a 3 token output language - space, no space, no text

    def __init__(self,
               *,
               num_layers,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               seq_len,
               dropout_rate=0.1
               ):
        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.pos_embedding = PostionalEmbedding(len(Decoder.output_tokens), d_model, seq_len)

        self.dec_layers = [
            DecoderLayer(
              d_model=d_model,
              num_attention_heads=num_attention_heads,
              dff=dff,
              dropout_rate=dropout_rate,
              name=f"decoder_sublayer_{i}")
            for i in range(num_layers)
        ]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def compute_mask(self, x, previous_mask=None):
        x = self.tokenizer(x)
        return self.pos_embedding.compute_mask(x, previous_mask)

    def call(self, dec_input, enc_output, enc_mask, training):
        mask = self.pos_embedding.compute_mask(dec_input)
        x = self.pos_embedding(dec_input)  # Shape: `(batch_size, target_seq_len, d_model)`.

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x  = self.dec_layers[i](x, mask, enc_output, enc_mask, training)

        # The shape of x is `(batch_size, target_seq_len, d_model)`.
        return x

class SpaceSegmentationTransformer(tf.keras.Model):
    """
    Transformer for finding space in english text without spaces

    Encoder takes in batches of strings, decoder takes in batches of seq_len
    tokens indicating space or not.

    """
    def __init__(self,
               *,
               num_layers, # Number of decoder layers.
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               input_tokenizer,
               seq_len,
               dropout_rate=0.1,
               classification_threshold=0.5,
               num_classes=2,
               pos_multiplier=1,
               ):
        super().__init__()
        d_model = d_model + 1 if d_model % 2 == 1 else d_model # Ensure even dimensionality so our positional encodings work
        self.encoder = Encoder(
          num_layers=num_layers,
          d_model=d_model,
          num_attention_heads=num_attention_heads,
          dff=dff,
          tokenizer=input_tokenizer,
          dropout_rate=dropout_rate,
          seq_len=seq_len,
          pos_multiplier=pos_multiplier
          )
        self.tokenizer = input_tokenizer
        self.dense = tf.keras.layers.Dense(num_classes)  # Why does softmax here break it?

    def call(self, inputs, training=False):
        """
        Expects inputs of text to be segmentated and, if training, segmentation
        labels aligned to input text - where 2 indicates space, 1 indicates 
        no space and 0 indicates a missing character (essentially end token).

        Assumes batch first ranks
        """
        to_enc = inputs
        run_sequential = False

        if not training:
            # This is where we will migrate sequential generation once we re-include the decoder
            batch_size = tf.shape(to_enc)[0]
            to_dec = tf.zeros((batch_size, 1))
            run_sequential = True

        
        # The encoder output.
        enc_output = self.encoder(to_enc, training)  # `(batch_size, inp_seq_len, d_model)`
        # enc_mask = self.encoder.compute_mask(to_enc)
        return tf.reshape(self.dense(enc_output), [tf.shape(inputs)[0], SEQ_LEN])

class LossWithVoids(tf.keras.losses.Loss):
    """
    Class to intercept any loss calculations when we want to ensure 'void'
    tokens in y_true are not included in loss calculations
    """
    def __init__(self, loss: tf.keras.losses.Loss, void_tokens: list, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss = loss
        self.void_tokens = void_tokens

    def __call__(self, y_true, y_pred, sample_weight=None):
        mask = tf.cast(tf.ones_like(y_true), tf.bool)
        for token in self.void_tokens:
            mask &= tf.not_equal(y_true, token) # (Batch, seq_len)
        return self.loss.__call__(y_true[mask], y_pred[mask]) 

class WarmupSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=200):
        super().__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return 2* tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

# Transformer Model

In [9]:
tokenizer = tf.keras.layers.TextVectorization(
    output_sequence_length=SEQ_LEN,
    standardize="lower_and_strip_punctuation",
    split="character",
    output_mode="int",
)
tokenizer.adapt(train_ds.take(10).map(lambda x,y: x))

In [10]:
NUM_HEADS = 2
KEY_DIM = 16
FF_DIM = 32
DROPOUT_RATE = 0.1
UNITS = 20

transformer = SpaceSegmentationTransformer(
    seq_len=SEQ_LEN,
    num_layers=1,
    d_model=32,
    num_attention_heads=NUM_HEADS,
    dff=FF_DIM,
    input_tokenizer=tokenizer,
    num_classes=1,
    dropout_rate=DROPOUT_RATE
)

In [11]:
# Test

transformer_out = transformer(test_inputs)
transformer_out.shape

2023-03-02 14:55:19.189906: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


TensorShape([128, 10])

In [12]:
transformer_out._keras_mask

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute '_keras_mask'

In [13]:
def masked_binary_crossentropy(y_true, y_pred):
  """
  y_true: shape (batch_size, seq_len). 1. = no space, 2. = space
  y_pred: shape (batch_size, seq_len, 1). Logits
  """
  labels = y_true  # 0 = no space, 1 = space

  # Deal with Keras 'feature' that squeezes out the last dimension silently (WTF)
  # if tf.shape(y_pred)[-1] == 1:  
  #   logits = tf.squeeze(y_pred, axis=-1)  # (batch_size, seq_len)
  # else:
  logits = y_pred
  probs = tf.nn.sigmoid(logits)
  bce = - labels * tf.math.log(probs) - ((1 - labels) * tf.math.log(1 - probs))

  return tf.reduce_mean(bce)

In [14]:
# Test

masked_binary_crossentropy(test_outputs, transformer_out)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5822634>

In [15]:
for test_inputs, test_outputs in train_ds.take(1):
  y_pred = transformer(test_inputs)
  loss = masked_binary_crossentropy(test_outputs, y_pred)

loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.5924214>

In [16]:
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC

earlystopping = EarlyStopping(patience=2, monitor="val_binary_accuracy")
transformer.compile(loss=masked_binary_crossentropy, optimizer='adam',
                    metrics=[BinaryAccuracy(), Precision(), Recall(), AUC(curve='PR')])

history = transformer.fit(train_ds, validation_data=valid_ds, epochs=20, callbacks=[earlystopping])

Epoch 1/20
79/79 [==============================] - 4s 30ms/step - loss: 0.3694 - binary_accuracy: 0.7913 - precision: 0.6296 - recall: 0.1876 - auc: 0.4344 - val_loss: 0.2833 - val_binary_accuracy: 0.8321 - val_precision: 0.7254 - val_recall: 0.4164 - val_auc: 0.6149
Epoch 2/20
79/79 [==============================] - 2s 23ms/step - loss: 0.2675 - binary_accuracy: 0.8506 - precision: 0.7440 - recall: 0.5170 - auc: 0.6447 - val_loss: 0.1904 - val_binary_accuracy: 0.8986 - val_precision: 0.8510 - val_recall: 0.6698 - val_auc: 0.7873
Epoch 3/20
79/79 [==============================] - 2s 24ms/step - loss: 0.1903 - binary_accuracy: 0.9071 - precision: 0.8386 - recall: 0.7299 - auc: 0.7797 - val_loss: 0.1038 - val_binary_accuracy: 0.9645 - val_precision: 0.8934 - val_recall: 0.9577 - val_auc: 0.9307
Epoch 4/20
79/79 [==============================] - 2s 24ms/step - loss: 0.1387 - binary_accuracy: 0.9385 - precision: 0.8925 - recall: 0.8275 - auc: 0.8498 - val_loss: 0.0415 - val_binary_accu

In [17]:
transformer.evaluate(test_ds, return_dict=True)

79/79 [==============================] - 1s 9ms/step - loss: 0.0109 - binary_accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 1.0000


{'loss': 0.01093901228159666,
 'binary_accuracy': 1.0,
 'precision': 1.0,
 'recall': 1.0,
 'auc': 1.0}

In [18]:
transformer(['ababab'+'b'*(SEQ_LEN-6)])

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-8.061153 ,  6.1619864, -8.910204 ,  4.308545 , -8.751168 ,
         4.4881577, -3.1239035, -6.4980745, -6.1512465, -4.930309 ]],
      dtype=float32)>

: 